# Geothermal Datathon 2021
---

<img src="../figures/the_roaring_kitties_logo.png" width="200" align="center">

Team members:
- **Ricardo Lara**
- **Artur Davletshin**
- **Aigul Akberova**
- **Sercan Gul**
- **Hakki Aydin**
- **Jose Hernandez**


---

### Import Modules

In [22]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 1 Eaglebine

In [29]:
directory = "../1 Data Exploration and Visualization/tidy data/"
EBt = pd.read_csv(directory + 'Eaglebine_Temperatures.csv')
EBmisc = pd.read_csv(directory + 'Eaglebine Misc.csv')
EBwl = pd.read_csv('imputed well logs/Eaglebine_imputed_well_logs.csv')

In [30]:
EB = EBt.merge(EBmisc, on='UWI', how='left')

In [31]:
EB2 = EB.merge(EBwl, on='UWI', how='left')

In [32]:
EB2.head()

,UWI,SurfLat,SurfLong,"TD, m_x","GL, m","Depth subsea, m","Depth subsurface, m","TSC, h","BHT, C","Synthetic Temp, C",...,cumwater,Unnamed: 0,GR,NPHI_SS,NPHI_LS,DPHI_SS,DPHI_LS,PEFZ,RHOB,CAL
0,42013301410000,28.690426,-98.470138,5982.919009,105.430317,3725.417881,3830.848197,5.50,61.666667,141.773056,...,NaN,0,28.919000,0.083307,-0.065809,-0.054688,-0.035348,4.951953,2.7783,6.0281
1,42013301930000,28.779921,-98.208313,3284.219895,121.127516,3168.700699,3289.828215,11.00,107.222222,132.005556,...,1002330.677,1,70.178291,0.299055,0.232384,0.194271,0.206172,4.846338,2.3503,7.0612
2,42013302760000,28.994360,-98.421799,1880.006340,140.329916,1747.418344,1887.748260,5.50,65.555556,88.974111,...,0.000,2,22.447700,0.153019,0.011500,0.004990,-0.009300,4.784638,2.7198,8.0507
3,42013305480000,28.759118,-98.156410,2498.750320,125.089916,2379.268724,2504.358640,5.00,76.666667,108.009111,...,0.000,3,38.467900,0.171462,0.044400,0.027966,0.030800,4.822343,2.6534,8.0486
4,42013310190000,28.817400,-98.155319,3352.799893,155.118811,3203.289401,3358.408213,7.75,98.888889,133.797009,...,NaN,4,25.944800,0.149961,0.011175,-0.014248,0.001377,4.893333,2.7101,7.5611


In [37]:
EB2.columns.str.match("Unnamed")
EB3 = EB2.loc[:,~EB2.columns.str.match("Unnamed")]
EB3.head()

,UWI,SurfLat,SurfLong,"TD, m_x","GL, m","Depth subsea, m","Depth subsurface, m","TSC, h","BHT, C","Synthetic Temp, C",...,cumgas,cumwater,GR,NPHI_SS,NPHI_LS,DPHI_SS,DPHI_LS,PEFZ,RHOB,CAL
0,42013301410000,28.690426,-98.470138,5982.919009,105.430317,3725.417881,3830.848197,5.50,61.666667,141.773056,...,NaN,NaN,28.919000,0.083307,-0.065809,-0.054688,-0.035348,4.951953,2.7783,6.0281
1,42013301930000,28.779921,-98.208313,3284.219895,121.127516,3168.700699,3289.828215,11.00,107.222222,132.005556,...,8702967.000,1002330.677,70.178291,0.299055,0.232384,0.194271,0.206172,4.846338,2.3503,7.0612
2,42013302760000,28.994360,-98.421799,1880.006340,140.329916,1747.418344,1887.748260,5.50,65.555556,88.974111,...,7.992,0.000,22.447700,0.153019,0.011500,0.004990,-0.009300,4.784638,2.7198,8.0507
3,42013305480000,28.759118,-98.156410,2498.750320,125.089916,2379.268724,2504.358640,5.00,76.666667,108.009111,...,10067.000,0.000,38.467900,0.171462,0.044400,0.027966,0.030800,4.822343,2.6534,8.0486
4,42013310190000,28.817400,-98.155319,3352.799893,155.118811,3203.289401,3358.408213,7.75,98.888889,133.797009,...,NaN,NaN,25.944800,0.149961,0.011175,-0.014248,0.001377,4.893333,2.7101,7.5611


In [34]:
EB3.columns

Index(['UWI', 'SurfLat', 'SurfLong', 'TD, m_x', 'GL, m', 'Depth subsea, m',
       'Depth subsurface, m', 'TSC, h', 'BHT, C', 'Synthetic Temp, C',
       'Static Temp, C', 'True Temp, C', 'source', 'label',
       'SurfaceLatitude_NAD27', 'SurfaceLongitude_NAD27',
       'BottomLatitude_NAD27', 'BottomLongitude_NAD27', 'oil total cum, bbl',
       'gas total cum, mcf', 'water total cum, bbl', 'gor total average',
       'gas maximum, mcf', 'oil maximum, bbl', 'water maximum, bbl', 'status',
       'Mud Wt', 'TD, m_y', 'MW at Depth(KB), m', 'KB, m',
       'Min casing size, in', 'cumoil', 'cumgas', 'cumwater', 'GR', 'NPHI_SS',
       'NPHI_LS', 'DPHI_SS', 'DPHI_LS', 'PEFZ', 'RHOB', 'CAL'],
      dtype='object')

In [16]:
EB3.to_csv('merged data/Eaglebine_merged.csv')

# Duvernay

In [17]:
directory = "../1 Data Exploration and Visualization/tidy data/"
DVt = pd.read_csv(directory + 'Duvernay_Temperatures.csv')
DVmisc = pd.read_csv(directory + 'Duvernay Misc.csv')
DVwl = pd.read_csv('imputed well logs/Duvernay_imputed_well_logs.csv')

In [18]:
DV = DVt.merge(DVmisc, on='UWI', how='left')
DV2 = DV.merge(DVwl, on='UWI', how='left')

DV2.columns.str.match("Unnamed")
DV3 = DV2.loc[:,~DV2.columns.str.match("Unnamed")]
DV3.head()

,UWI,"Depth subsea, m","BHT, C",Formation,"Synthetic Temp, C","Static Temp, C","True Temp, C",source,label,SurfaceLatitude_NAD27,...,"Maximum Shut-in Pressure, kPa","Final Hydrostatic Pressure, kPa",GR,NPHI_SS,NPHI_LS,DPHI_SS,DPHI_LS,PEFZ,RHOB,CAL
0,100010107020W500,716.0,48.33,TRmontney,60.97,NaN,60.97,synthetic,train,55.027981,...,12858.7,15899.31,1.14200,0.126523,0.056289,0.059455,0.079798,3.587307,30.175647,7.5523
1,100010608109W500,1044.4,34.26,Dmuskeg,63.98,NaN,63.98,synthetic,train,55.987511,...,10782.0,18137.00,21.87750,0.111200,-0.001340,-0.086958,-0.092600,4.162300,2873.401100,206.7875
2,100010904012W500,2424.2,104.40,Melkton,NaN,NaN,NaN,NaN,not assigned,52.424180,...,47675.0,NaN,42.45040,0.149962,0.057600,-0.094885,-0.082800,4.707043,2851.595500,211.8617
3,100011206021W400,171.9,29.00,Dwintrbrn,39.07,NaN,39.07,synthetic,train,54.167413,...,5317.0,6603.00,28.45550,0.274898,0.159300,0.051155,0.070900,3.504784,2614.128200,202.6417
4,100011506915W500,936.4,64.00,Dwabamun,69.47,NaN,69.47,synthetic,train,54.969037,...,13781.0,20357.00,5.14509,0.175245,0.112700,0.205905,0.195500,1.867400,2374.466300,207.7325


In [20]:
DV3.to_csv('merged data/Duvernay_merged.csv')

# Impute missing data

In [21]:
def iterative_imputer(df_original, columns):
    df = df_original[columns].copy(deep=True)
    imputer = IterativeImputer(random_state=42)
    
    df = pd.DataFrame(imputer.fit_transform(df), columns =df.columns)

    df2 = df_original.copy(deep=True)
    df2[columns] = df.values
    return df2

In [38]:
EB3.isnull().sum()

UWI                         0
SurfLat                     0
SurfLong                    0
TD, m_x                     0
GL, m                       0
Depth subsea, m             0
Depth subsurface, m         0
TSC, h                      0
BHT, C                      0
Synthetic Temp, C          81
Static Temp, C            305
True Temp, C               81
source                     81
label                       0
SurfaceLatitude_NAD27      16
SurfaceLongitude_NAD27     16
BottomLatitude_NAD27       16
BottomLongitude_NAD27      16
oil total cum, bbl         78
gas total cum, mcf         78
water total cum, bbl       78
gor total average          86
gas maximum, mcf           78
oil maximum, bbl           78
water maximum, bbl         78
status                     16
Mud Wt                      8
TD, m_y                     8
MW at Depth(KB), m          8
KB, m                       8
Min casing size, in        57
cumoil                     74
cumgas                     74
cumwater  

In [39]:
cols = ['TD, m_x',
'GL, m',
'SurfLat',                     
'SurfLong',               
'SurfaceLatitude_NAD27',  
'SurfaceLongitude_NAD27', 
'BottomLatitude_NAD27',   
'BottomLongitude_NAD27',  ]

EB4 = iterative_imputer(EB3, cols)

In [44]:
cols = [
'TD, m_x',                    
'BottomLatitude_NAD27',       
'BottomLongitude_NAD27',      
'oil total cum, bbl',         
'gas total cum, mcf',         
'water total cum, bbl',       
'gor total average',          
'gas maximum, mcf',           
'oil maximum, bbl',           
'water maximum, bbl',         
'cumoil',                     
'cumgas',                     
'cumwater',                   
]

EB5 = iterative_imputer(EB4, cols)

In [45]:
cols = [
'TD, m_x',
'CAL',
'Min casing size, in'
]

EB6 = iterative_imputer(EB5, cols)

In [46]:
cols = [
'TD, m_x',                    
'BottomLatitude_NAD27',       
'BottomLongitude_NAD27',      
'Mud Wt',                      
'MW at Depth(KB), m',                   
]

EB7 = iterative_imputer(EB6, cols)

In [58]:
EB7.columns.to_list()

['UWI',
 'SurfLat',
 'SurfLong',
 'TD, m_x',
 'GL, m',
 'Depth subsea, m',
 'Depth subsurface, m',
 'TSC, h',
 'BHT, C',
 'Synthetic Temp, C',
 'Static Temp, C',
 'True Temp, C',
 'source',
 'label',
 'SurfaceLatitude_NAD27',
 'SurfaceLongitude_NAD27',
 'BottomLatitude_NAD27',
 'BottomLongitude_NAD27',
 'oil total cum, bbl',
 'gas total cum, mcf',
 'water total cum, bbl',
 'gor total average',
 'gas maximum, mcf',
 'oil maximum, bbl',
 'water maximum, bbl',
 'status',
 'Mud Wt',
 'TD, m_y',
 'MW at Depth(KB), m',
 'KB, m',
 'Min casing size, in',
 'cumoil',
 'cumgas',
 'cumwater',
 'GR',
 'NPHI_SS',
 'NPHI_LS',
 'DPHI_SS',
 'DPHI_LS',
 'PEFZ',
 'RHOB',
 'CAL']

# 1 Duvernay

In [48]:
DV.isnull().sum()

UWI                                  0
Depth subsea, m                      0
BHT, C                               0
Formation                            0
Synthetic Temp, C                  140
Static Temp, C                     490
True Temp, C                       140
source                             140
label                                0
SurfaceLatitude_NAD27                0
SurfaceLongitude_NAD27               0
BottomLatitude_NAD27                 0
BottomLongitude_NAD27                0
oil total cum, bbl                 357
gas total cum, mcf                 357
water total cum, bbl               357
gor total average                  372
gas maximum, mcf                   357
oil maximum, bbl                   357
water maximum, bbl                 357
yield total average                359
status                               0
Start TVD, m                       479
End TVD, m                         479
Start MD, m                         30
End MD, m                

In [49]:
DV.isnull().sum()

UWI                                  0
Depth subsea, m                      0
BHT, C                               0
Formation                            0
Synthetic Temp, C                  140
Static Temp, C                     490
True Temp, C                       140
source                             140
label                                0
SurfaceLatitude_NAD27                0
SurfaceLongitude_NAD27               0
BottomLatitude_NAD27                 0
BottomLongitude_NAD27                0
oil total cum, bbl                 357
gas total cum, mcf                 357
water total cum, bbl               357
gor total average                  372
gas maximum, mcf                   357
oil maximum, bbl                   357
water maximum, bbl                 357
yield total average                359
status                               0
Start TVD, m                       479
End TVD, m                         479
Start MD, m                         30
End MD, m                

In [52]:
cols = [
'Depth subsea, m',                    
'BottomLatitude_NAD27',       
'BottomLongitude_NAD27',      
'oil total cum, bbl',         
'gas total cum, mcf',         
'water total cum, bbl',       
'gor total average',          
'gas maximum, mcf',           
'oil maximum, bbl',           
'water maximum, bbl',         
'yield total average',
'Maximum Shut-in Pressure, kPa'                  
]

DV2 = iterative_imputer(DV, cols)

In [54]:
DV2['DST BHT, C'].mask(DV2['DST BHT, C'].isnull(), DV2['BHT, C'], inplace=True)


In [56]:
DV3 = DV2.drop([
'Start TVD, m',
'End TVD, m', 
'Start MD, m',
'End MD, m',
'Pressure Recorder Depth, m',
'Final Hydrostatic Pressure, kPa'    ], axis=1)

In [57]:
DV3.to_csv('merged data/Duvernay_merged.csv', index=False)